In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Etape 1:
1/ Importation des données.
2/ "URL", "RESPONSE", "HTML" permettent de récupérer le texte du site.
3/ "SOUP" permet d'utiliser les données pour les scraper.

In [2]:
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
data = soup.find_all('div')

### Etape 2:
1/ Création du dataframe "df1" recueillant toutes les données.
1.1/ "a" est le dataframe pour la phase de la compétition.
1.2/ "b" est le dataframe pour la compétition.
2/ Les deux dataframes "a" et "b" sont fusionnés dans le df1.

In [3]:
df1 = pd.DataFrame()

a = pd.DataFrame(columns=["1"])
b = pd.DataFrame(columns=["2"])

col = soup.find_all('div', class_='match-item-event-series text-of')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    a = a._append({'1':text1}, ignore_index=True)
col1 = soup.find_all('div', class_='match-item-event text-of')
for cols in col1:
    texts1 = cols.find_all(string=True, recursive=False)
    text2 = texts1[1].strip()
    b = b._append({'2':text2}, ignore_index=True)

df1 = b.join(a, how='inner')
df1

,2,1
0,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals
1,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals
2,Multiplatform Esports Game 2023,Playoffs–Lower Final
3,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6
4,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6
5,China Evolution Series Act 2: Selection,Group Stage–Winner's (B)
6,China Evolution Series Act 2: Selection,Group Stage–Winner's (A)
7,Predator League Japan 2024,Main Event–Grand Final
8,Liquid Open 2023: Oceania,Playoffs–Grand Final
9,Predator League Japan 2024,Main Event–Consolation Final


In [4]:
soup.find_all('div', class_='match-item-vs-team')

[<div class="match-item-vs-team mod-winner">
 <div class="match-item-vs-team-name">
 <i class="js-spoiler fa fa-caret-right" style="position: absolute; left: -10px; top: 1px; color: #61b875; font-size: 11px;"></i>
 <div class="text-of">
 <span class="flag mod-us"></span>
 											Argon Black										</div>
 </div>
 <div class="match-item-vs-team-score js-spoiler">
 																					2																			</div>
 </div>,
 <div class="match-item-vs-team">
 <div class="match-item-vs-team-name">
 <div class="text-of">
 <span class="flag mod-us"></span>
 											Square Nation										</div>
 </div>
 <div class="match-item-vs-team-score js-spoiler">
 																					1																			</div>
 </div>,
 <div class="match-item-vs-team mod-winner">
 <div class="match-item-vs-team-name">
 <i class="js-spoiler fa fa-caret-right" style="position: absolute; left: -10px; top: 1px; color: #61b875; font-size: 11px;"></i>
 <div class="text-of">
 <span class="flag mod-us"></span>
 								